In [25]:
import pandas as pd
import re
import numpy as np

In [26]:
template_questions_dataframe = pd.read_csv('../training_data/Question_Answer_NLI.csv')
template_questions_dataframe = template_questions_dataframe.replace(np.nan, '', regex=True)
NUMBER_OF_ROWS = 175

In [27]:
problem_list = [{"PROBLEM_1":"Diabetes"},{"PROBLEM_1":"heart failure"}]
treatment_list = [{"TREATMENT_1":"Januvia"},{"TREATMENT_1":"Digoxin"}]
religion_list = [{"RELIGION_1":"BUDDHIST"},{"RELIGION_1":"CATHOLIC"}]
insurance_list = [{"INSURANCE_1":"Medicare"},{"INSURANCE_1":"Medicaid"}]
ethinicity_list = [{"ETHINICITY_1":"BLACK"},{"ETHINICITY_1":"ASIAN"}]
numeric_list = [{"NUMERIC_[0-9]":"1.5"},{"NUMERIC_[0-9]":"4.9"},{"NUMERIC_[0-9]":"3.54"}]
test_list = [{"TEST_1":"radiograph"}]
problem_and_treatment_list = [{"PROBLEM_1":"Diabetes","TREATMENT_1":"Insulin"}]
treatment_and_treatment_list = [{"TREATMENT_1":"Tamoxifen","TREATMENT_2":"Warfarin"}]
ethinicity_and_problem_list = [{"ETHINICITY_1":"BLACK","PROBLEM_1":"immunodeficiency"}]
problem_and_test_list = [{"PROBLEM_1":"immunodeficiency","TEST_1":"HIV"}]
numeric_and_numeric_list = [{"NUMERIC_1":"10","NUMERIC_2":"49"},{"NUMERIC_1":"30","NUMERIC_2":"79"}]

In [28]:
def replace(nl_tem, sql_tem, dict_list=None):
    nl_output = []
    sql_output = []
    
    if dict_list is None:
        nl_output.append(nl_tem)
        sql_output.append(sql_tem)
        return nl_output, sql_output
    
    for _dict in dict_list:
        nl = nl_tem
        sql = sql_tem
        for key, value in _dict.iteritems():
            nl = re.sub(key, value, nl)
            sql = re.sub(key, value, sql)
        nl_output.append(nl)
        sql_output.append(sql)
    return nl_output, sql_output

def get_result(nl_tem, sql_tem, nl_output_list, sql_output_list):
    result = {'nl_template' : [],
              'nl' : [],
              'sql': [],
              'sql_template':[]}
    for index in range(len(nl_output_list)):
        result['nl_template'].append(nl_tem)
        result['sql_template'].append(sql_tem)
        result['nl'].append(nl_output_list[index])
        result['sql'].append(sql_output_list[index])
    return result

        
def fill_template(nl, sql):
    nl_list = [nl]
    sql_list = [sql]
    
    for i in range(2):
        final_nl_output = []
        final_sql_output = []
        for index in range(len(nl_list)):
            nl = nl_list[index]
            sql = sql_list[index]
            if "PROBLEM_1" in nl and "TREATMENT_1" in nl:
                nl_output, sql_output = replace(nl, sql, problem_and_treatment_list)
            elif "TREATMENT_1" in nl and "TREATMENT_2" in nl:
                nl_output, sql_output = replace(nl, sql, treatment_and_treatment_list)
            elif "ETHINICITY_1" in nl and "PROBLEM_1" in nl:
                nl_output, sql_output = replace(nl, sql, ethinicity_and_problem_list)
            elif "PROBLEM_1" in nl and "TEST_1" in nl:
                nl_output, sql_output = replace(nl, sql, problem_and_test_list)
            elif "NUMERIC_1" in nl and "NUMERIC_2" in nl:
                nl_output, sql_output = replace(nl, sql, numeric_and_numeric_list)
            elif "PROBLEM_1" in nl:
                nl_output, sql_output = replace(nl, sql, problem_list)
            elif "TREATMENT_1" in nl:
                nl_output, sql_output = replace(nl, sql, treatment_list)
            elif "RELIGION_1" in nl:
                nl_output, sql_output = replace(nl, sql, religion_list)
            elif "INSURANCE_1" in nl:
                nl_output, sql_output = replace(nl, sql, insurance_list)
            elif "ETHINICITY_1" in nl:
                nl_output, sql_output = replace(nl, sql, ethinicity_list)
            elif "NUMERIC_" in nl:
                nl_output, sql_output = replace(nl, sql, numeric_list)
            elif "TEST_1" in nl:
                nl_output, sql_output = replace(nl, sql, test_list)         
            else:
                nl_output, sql_output = replace(nl, sql)
            
            final_nl_output.extend(nl_output)
            final_sql_output.extend(sql_output)
        nl_list = final_nl_output
        sql_list = final_sql_output
    return nl_list, sql_list

In [29]:
result = {'nl_template' : [],
          'nl' : [],
          'sql': [],
          'sql_template':[]}
output_dataframe = pd.DataFrame.from_dict(result)

output_questions_file = open('../training_data/mimic_tr.nl.tem','w')
output_sql_file = open('../training_data/mimic_tr.sql.tem','w')

for index, row in template_questions_dataframe.iterrows():
    nl_template, sql_template = row['nl_template'], row['sql_template']
    
    sql_template = sql_template.replace('\n','')
    
    output_questions_file.write(nl_template)
    output_questions_file.write("\n")
    output_sql_file.write(sql_template)
    output_sql_file.write("\n")
    
    nl_output_list, sql_output_list = fill_template(nl_template, sql_template)
    result = get_result(nl_template, sql_template, nl_output_list, sql_output_list)
    result = pd.DataFrame.from_dict(result)
    output_dataframe = pd.concat([output_dataframe, result])

    if index > NUMBER_OF_ROWS:
        break

output_questions_file.close()
output_sql_file.close()

In [71]:
output_dataframe.to_csv('../training_data/final.csv', index=False)

In [69]:
output_dataframe

,nl,nl_template,sql,sql_template
0,How many patients suffer from Diabetes,How many patients suffer from PROBLEM_1,select count(distinct subject_id) from DIAGNOS...,select count(distinct subject_id) from DIAGNOS...
1,How many patients suffer from heart failure,How many patients suffer from PROBLEM_1,select count(distinct subject_id) from DIAGNOS...,select count(distinct subject_id) from DIAGNOS...
0,How many patients have Diabetes,How many patients have PROBLEM_1,select count(distinct subject_id) from DIAGNOS...,select count(distinct subject_id) from DIAGNOS...
1,How many patients have heart failure,How many patients have PROBLEM_1,select count(distinct subject_id) from DIAGNOS...,select count(distinct subject_id) from DIAGNOS...
0,How many Diabetes patients have been admitted,How many PROBLEM_1 patients have been admitted,select count(distinct subject_id) from DIAGNOS...,select count(distinct subject_id) from DIAGNOS...
1,How many heart failure patients have been admi...,How many PROBLEM_1 patients have been admitted,select count(distinct subject_id) from DIAGNOS...,select count(distinct subject_id) from DIAGNOS...
0,How many patients have been administered Januvia,How many patients have been administered TREAT...,select count(distinct subject_id) from PRESCRI...,select count(distinct subject_id) from PRESCRI...
1,How many patients have been administered Digoxin,How many patients have been administered TREAT...,select count(distinct subject_id) from PRESCRI...,select count(distinct subject_id) from PRESCRI...
0,How many patients have been given Januvia,How many patients have been given TREATMENT_1,select count(distinct subject_id) from PRESCRI...,select count(distinct subject_id) from PRESCRI...
1,How many patients have been given Digoxin,How many patients have been given TREATMENT_1,select count(distinct subject_id) from PRESCRI...,select count(distinct subject_id) from PRESCRI...


In [19]:
len(output_dataframe)

434